# Lab 1: scRNA-seq Data Preprocessing

## Part 1: Understanding Barcodes

### References

Geneious Biologics — "Understanding Single Cell technologies: Barcodes and UMIs" (https://help.geneiousbiologics.com/hc/en-us/articles/4781289585300-Understanding-Single-Cell-technologies-Barcodes-and-UMIs)

Kebschull, J.M., and Zador, A.M. (2018). Cellular barcoding: lineage tracing, screening and beyond. Nat. Methods 15, 871–879. https://doi.org/10.1038/s41592-018-0185-x.




### Background

#### Single Cell Overview
Single Cell technologies like 10X Genomics and BD Rhapsody allow you to partition individual cells into wells or droplets and sequence the mRNA reads of the individual cells. Because of this, the sequencing output is often referred to as an Expression Library. The technology relies on the capture of the mRNA poly-A tail using a poly dT sequence that is attached to a bead.

<img src="https://help.geneiousbiologics.com/hc/article_attachments/8234399911316" alt="beads" width="500"/>

As can be seen above, there are two other features found on the sequence used to capture mRNAs: a Barcode and a UMI. These are crucial to understanding your sequence library and evaluating the quality of your reads. In general, Single Cell technologies generate paired-end reads that have a Barcode followed by a UMI at either the 3' or 5' end of the parent sequence.

#### What are Barcodes?

There are two broad types of Barcodes: Feature Barcodes and Cell Barcodes. Both are short nucleotide tags (~16 bp) that are used to "label" sequences. These labels are used to sort sequences that have originated from a single cell source, or to denote some other feature of the cell like the presence of a particular cell surface protein.

**Cell Barcode:** Sequences with the same cell barcode can be grouped together as coming from the same cell source.

**Feature Barcode:** Used as an additional tag to indicate the presence of a cell surface protein. As seen below, the Feature Barcode is concatenated to an antibody against the cell surface target. The Feature Barcode can then be assigned to a Cell Barcode via capture to a bead containing the Cell Barcode.

<img src="https://help.geneiousbiologics.com/hc/article_attachments/8403663699476" alt="feature barcode" width="500"/>

#### What are UMIs?
UMIs (Unique Molecular Identifiers) are used for quality control purposes and are also used in other non-Single Cell sequencing kits. Like Barcodes, UMIs are short nucleotide sequences. However, unlike Barcodes the sequences are random with a very low likelihood of a duplicate UMI within a single bead; therefore each mRNA read is assigned a unique UMI.

As the starting material is very low and of poor quality, PCR amplification is needed to generate enough material for High Throughput Sequencing technologies like Illumina. During PCR amplification, base incorporation errors and biased amplification can occur. These stochastic effects in the first rounds of PCR can propagate through to the final sequenced library. UMIs are used to screen out these errors.


##### Quantitative Analysis

The below figure shows how UMIs can be used to determine the starting mRNA content, as each UMI represents one initial mRNA read.

<img src="https://help.geneiousbiologics.com/hc/article_attachments/12808324089236" alt="UMI quantification" width="500"/>

### Final Read Format

The below image provides an overview of a generic initial amplification process. Upon reaching the 5’ end of the RNA template during first-strand synthesis, the reverse transcriptase enzyme used in the reaction appends additional non-template nucleotides to the sequence, mostly deoxycytidine residues. This non-template sequence is used as a binding site for a template switching oligonucleotide (TSO). The TSO causes the transcriptase enzyme to switch from transcribing the RNA template sequence to transcribing the TSO sequence, generating the complementary cDNA strand.

<img src="https://help.geneiousbiologics.com/hc/article_attachments/8404901907988" alt="final read format" width="500"/>

Single-cell RNA-seq workflows use two short sequences to turn sequencing reads into accurate per-cell molecule counts: a cell barcode (which labels the droplet or well a read came from) and a UMI (unique molecular identifier) that tags individual RNA molecules before amplification. Together they let pipelines group reads by cell and collapse PCR duplicates so counts reflect input molecules rather than amplification artifacts.




### tl;dr

+ **Barcode placement and format depend on the technology** (for example, many droplet-based methods place cell barcodes and UMIs at the start of Read 1 or in a dedicated index read). Sequencing strategies separate the short barcode/UMI reads from the longer cDNA read.

**_Determine the barcode and UMI structure - ASK!!_**

+ **Cell barcodes may be drawn from a predefined set - _whitelist_ for many platforms** this lets tools correct sequencing errors by matching to the nearest valid barcode. Barcode lengths are protocol-specific (10x commonly uses a 16 bp cell barcode).
UMIs are short random sequences (10–12 bp is common) attached to molecules before amplification; they allow deduplication of PCR duplicates by collapsing reads that share the same UMI and map to the same gene.

**Common issues**
+ sequencing errors in barcodes/UMIs
+ collisions (different molecules sharing the same UMI by chance)
+ ambient RNA and multiplets (droplets containing more than one cell)

## Exercise 1: Finding single-cell datasets


### Key metadata for each dataset:

+ Sample information
    + Organism
    + Tissue type
    + Condition (e.g., healthy vs. diseased)
    + Cells vs. nuclei
+ Technology used (e.g., 10x Genomics Chromium)
    + 10x instrument
    + Product (e.g., Single Cell 3' GEX)
    + Chemistry version (e.g., 3.1)
+ Number of cells / target number of cells

### Possible sources of single-cell datasets:

#### 10x Genomics
Datasets created by 10x Genomics are available on the 10x genomics website

[10x Datasets](https://www.10xgenomics.com/resources/datasets)


#### Human Cell Atlas (HCA)

The Human Cell Atlas (HCA) is an international effort to create comprehensive reference maps of all human cells. The HCA Data Portal provides access to a wide range of single-cell datasets across various tissues and conditions. You can explore the HCA Data Portal to find datasets that match your research interests.

[HCA Data Explorer](https://explore.data.humancellatlas.org/projects)


#### NCBI

The NCBI Sequence Read Archive (SRA) is a public repository for raw sequencing data, including single-cell RNA-seq datasets. You can search for single-cell datasets using keywords related to your area of interest (e.g., "single-cell RNA-seq", "scRNA-seq", "10x Genomics", etc.) and filter results by organism, tissue type, and other metadata.

[NCBI SRA](https://www.ncbi.nlm.nih.gov/sra)

NCBI GEO (Gene Expression Omnibus) is another repository that contains processed gene expression data, including single-cell RNA-seq datasets. You can search for datasets using similar keywords and filter results based on metadata.

[NCBI GEO](https://www.ncbi.nlm.nih.gov/geo/)


## Exercise 1: Hands-on

Search the above resources for a single-cell RNA-seq dataset that interests you. Make sure to note the key metadata for the dataset, including the sample information, technology used, and number of cells.

#### Questions
1. What is the sample information for the dataset you chose? (e.g., organism, tissue type, condition)
2. What technology was used to generate the dataset? (e.g., 10x Genomics Chromium, product, chemistry version)
3. How many cells were targeted in the dataset?
4. Were you able to find the raw sequencing data (FASTQ files) for the dataset? If so, where are they located?

#### Example datasets

<details>
  <summary>Click to expand/collapse this section</summary>

#### 10x Genomics

[500 Human PBMCs](https://www.10xgenomics.com/datasets/500-human-pbm-cs-3-lt-v-3-1-chromium-x-3-1-low-6-1-0)

#### Human Cell Atlas

[A cell atlas of human thymic development defines T cell repertoire formation](https://explore.data.humancellatlas.org/projects/c1810dbc-16d2-45c3-b45e-3e675f88d87b)

#### NCBI

[Human subcutaneous and visceral adipocyte atlases uncover classical and nonclassical adipocytes and depot-specific patterns](https://pubmed.ncbi.nlm.nih.gov/39856219/)

[Human subcutaneous and visceral adipocyte atlases](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE281356)
[Run Selector](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA1183538)



</details>

#### Download 10x Genomics dataset

In [ ]:
%%bash
if [[ ! -e ../../data/500_PBMC_3p_LT_Chromium_X_fastqs.tar ]]; then
    echo "Downloading 10x Genomics dataset..."
    (
        cd ../../data
        # Input Files
        curl -O https://cf.10xgenomics.com/samples/cell-exp/6.1.0/500_PBMC_3p_LT_Chromium_X/500_PBMC_3p_LT_Chromium_X_fastqs.tar
    )
fi
(
    cd ../../data &&  grep "500_PBMC_3p_LT_Chromium_X_fastqs.tar" checksums.md5 | md5sum -c -
)

#### Extracting FASTQ files from the downloaded archive

In [ ]:
%%bash
if [[ ! -e ../../data/500_PBMC_3p_LT_Chromium_X_fastqs/500_PBMC_3p_LT_Chromium_X_S4_L003_R1_001.fastq.gz ]]; then
    (
        cd ../../data && tar -xvf 500_PBMC_3p_LT_Chromium_X_fastqs.tar
    )
fi

##### Make example data

In [ ]:
%%bash
mkdir -p ../../sample_data/500_PBMC_3p_LT_Chromium_X_fastqs

(
    cd ../../sample_data/500_PBMC_3p_LT_Chromium_X_fastqs
    for f in ../../data/500_PBMC_3p_LT_Chromium_X_fastqs/*.fastq.gz; do
        nf=$(basename ${f%.*})
        gunzip -c $f | tail -n+1000001 | head -n 400 > $nf
    done
)

#### What is included in the dataset?

In [ ]:
%%bash

ls ../../data/500_PBMC_3p_LT_Chromium_X_fastqs

##### How can you tell what data is contained in each file? (Hint: look at the file names and the 10x Genomics documentation for this dataset)

In [ ]:
%%bash

peekfq () {
    local file="$1"
    if [[ "$file" == *.gz ]]; then
        gunzip -c "$file" | head -n4
    else
        head -n4 "$file"
    fi
}

for f in ../../data/500_PBMC_3p_LT_Chromium_X_fastqs/500_PBMC_3p_LT_Chromium_X_S4_L003_I1_001.fastq.gz; do
    echo "Peeking into $f"
    peekfq "$f"
done


#### Download and extract from SRA

In [ ]:
%%bash
accs="SRR31277936 SRR31277937"
for acc in $accs; do
    if [[ ! -e ../../data/$acc/$acc.sra ]]; then
        echo "Downloading $acc from NCBI SRA..."
        prefetch --output-directory ../../data $acc
    fi
    if [[ ! -e ../../data/$acc/${acc}_1.fastq.gz ]]; then
    parallel-fastq-dump\
     --sra-id ../../data/$acc/$acc.sra\
     --threads 8\
     --outdir ../../data/$acc\
     --split-3 -F --defline-qual '+'\
     --gzip
done


#### Make example data

In [ ]:
%%bash
accs="SRR31277936 SRR31277937"
for acc in $accs; do
    mkdir -p ../../sample_data/$acc
    if [[ ! -e ../../sample_data/$acc/${acc}_1.fastq ]]; then
        fastq-dump\
         --outdir ../../sample_data/$acc\
         --split-3 -F --defline-qual '+'\
         -N 1000001 -X 1000100\
         $acc
    fi
done

## Part 2: UMI Extraction

TODO: Implement UMI extraction and deduplication

In [ ]:
# Your code here

## Part 3: Quality Control

TODO: Implement quality metrics and filtering

In [ ]:
# Your code here